In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the CSV file
data = pd.read_csv('data-ga-en.csv')

In [2]:
print(data.head())

# Separate the columns into lists
twi_texts = data['ga'].tolist()
english_texts = data['english'].tolist()


                     ga          english
0  Bo ni otswa mi aloo?  Did you call me
1  Bo ni otswa mi aloo?  Did you call me
2  Bo ni otswa mi aloo?  Did you call me
3  Bo ni otswa mi aloo?  Did you call me
4  Bo ni otswa mi aloo?  Did you call me


In [3]:

# Preprocess the texts
def preprocess_texts(texts):
    texts = [text.lower() for text in texts]
    texts = ['<start> ' + text + ' <end>' for text in texts]
    return texts

twi_texts = preprocess_texts(twi_texts)
english_texts = preprocess_texts(english_texts)

# Tokenize the texts
twi_tokenizer = Tokenizer()
twi_tokenizer.fit_on_texts(twi_texts)
twi_sequences = twi_tokenizer.texts_to_sequences(twi_texts)

english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(english_texts)
english_sequences = english_tokenizer.texts_to_sequences(english_texts)

# Pad the sequences
max_twi_len = max([len(seq) for seq in twi_sequences])
max_english_len = max([len(seq) for seq in english_sequences])

twi_sequences = pad_sequences(twi_sequences, maxlen=max_twi_len, padding='post')
english_sequences = pad_sequences(english_sequences, maxlen=max_english_len, padding='post')

# Prepare the input and output sequences
encoder_input_data = np.array(twi_sequences)
decoder_input_data = np.array(english_sequences[:, :-1])
decoder_output_data = np.array(english_sequences[:, 1:])

AttributeError: 'float' object has no attribute 'lower'

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Define the input dimensions
twi_vocab_size = len(twi_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1
embedding_dim = 256
latent_dim = 512

# Define the encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=twi_vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(english_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


NameError: name 'twi_tokenizer' is not defined

In [5]:
batch_size = 64
epochs = 100

model.fit([encoder_input_data, decoder_input_data], 
          decoder_output_data, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_split=0.2)


NameError: name 'model' is not defined

In [ ]:
import pandas as pd

# Load the Twi text file
twi_data = pd.read_csv('twi.csv', header=None, names=['Twi'], delimiter=',', error_bad_lines=False)
# Fill missing values with an empty string
twi_data['Twi'] = twi_data['Twi'].fillna('')

# Load the English text file
english_data = pd.read_csv('english.csv', header=None, names=['English'], delimiter=',', error_bad_lines=False)
# Fill missing values with an empty string
english_data['English'] = english_data['English'].fillna('')

# Convert the data to lists
twi_texts = twi_data['Twi'].tolist()
english_texts = english_data['English'].tolist()
